In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
chop = Options()
chop.add_argument("headless=new")

driver = webdriver.Chrome(options=chop)


Get tables and keep the first one

In [2]:
from dataformat import ITableRow_US
import re
def get_numbers(string_item): 
    return re.sub("[^\d\.]", "" ,string_item)
ENTIDADES_URL = "https://es.wikipedia.org/wiki/Estado_de_los_Estados_Unidos"
driver.get(ENTIDADES_URL)
table = driver.find_elements(By.CSS_SELECTOR,"table")[1]
rows  = table.find_elements(By.CSS_SELECTOR, "tbody>tr")

all_rows = []


for row in rows:
    cells         = row.find_elements(By.CSS_SELECTOR,"td")
    name          = cells[1].text
    official_name = cells[2].text
    abbreviation  = cells[3].text
    density       = float(get_numbers(cells[-1].text))
    surface       = float(get_numbers(cells[-2].text))
    population    = int(get_numbers(cells[-3].text))

    table_row = ITableRow_US(name=name,
                            official_name=official_name,
                            abbreviation=abbreviation,
                            density=density,
                            surface=surface,
                            population=population)
    all_rows.append(table_row)


Writing this data using the

In [3]:
from datawriter import DataWriter

dw = DataWriter('./out.xlsx')
dw.write_us_table(all_rows)


In [4]:
rows  = table.find_elements(By.CSS_SELECTOR, "tbody>tr")
all_hrefs = []
for row in rows:
    # Get the second entry of the row
    cell = row.find_elements(By.CSS_SELECTOR, "td")[1]
    # Get the href link for that element
    href = cell.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    all_hrefs.append(href)


Use one of the hrefs to access the desired data. Similar to toponimias

In [5]:
from dataformat import IEtimilogia_US
etimologias = []

for link in all_hrefs:
    driver.get(link)

    text_area = driver.find_element(By.CSS_SELECTOR, "#mw-content-text>div")
    paragraphs = text_area.find_elements(By.XPATH, "./*")

    is_toponimia_next = False

    for paragraph in paragraphs:
        if paragraph.tag_name == "h2":
            try:
                titulo = paragraph.find_element(By.ID, "Etimología").get_attribute("innerHTML")
                is_toponimia_next = True
                continue
            except:
                pass
        if is_toponimia_next:
            # Avoid figures or spans embedded between title and text
            if paragraph.tag_name != "p":
                continue
            etimologias.append(IEtimilogia_US(link = link , text = paragraph.text))
            break
print(dtoponimias)



[IEtimilogia_US(link='https://es.wikipedia.org/wiki/Wyoming', text='El estado recibe el nombre del río Alabama, que lo atraviesa hasta desembocar cerca de Mobile. La palabra Alabama proviene del nombre de la tribu nativa choctaw que habitó el territorio en tiempos antiguos, estos se autodenominaban Alibamu que en inglés significa I open the thicket («yo elimino la maleza»).[cita requerida]'), IEtimilogia_US(link='https://es.wikipedia.org/wiki/Wyoming', text='La península de la Florida fue descubierta por Juan Ponce de León el lunes de Pascua de Resurrección del año 1513. Como la Pascua de Resurrección también se conoce como la Pascua Florida, a la península se le llamó: Península de la Florida.'), IEtimilogia_US(link='https://es.wikipedia.org/wiki/Wyoming', text='Idaho es quizás el único estado en ser nombrado prácticamente a consecuencia de un error. A comienzos de los años 1860, cuando el Congreso de Estados Unidos entreveía la posibilidad de administrar un territorio en las Montañas